In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig
import torch
model_name = "TheBloke/Tess-10.7B-v1.5b-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cuda:0", output_attentions=True).to("cuda")


In [2]:
# Load model directly
#from transformers import AutoTokenizer, AutoModelForCausalLM


from bertviz import head_view
from bertviz import model_view
from bertviz.neuron_view import show
# load the model
# Vicuna is an instruction-model based on Llama
#model_name = "TheBloke/Tess-10.7B-v1.5b-GPTQ" # mistralai/Mistral-7B-Instruct-v0.1
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name, output_attentions=True, device_map="auto")

input_sentence = """	
Neville: Hi there, does anyone remember what date I got married on? Don: Are you serious? Neville: Dead serious. We're on vacation, and Tina's mad at me about something. I have a strange suspicion that this might have something to do with our wedding anniversary, but I have nowhere to check. Wyatt: Hang on, I'll ask my wife. Don: Haha, someone's in a lot of trouble :D Wyatt: September 17. I hope you remember the year ;)
emotions"""
inputs = tokenizer.encode(input_sentence, return_tensors='pt').to("cuda")
outputs = model(inputs)
attention = outputs[-1]  # Output includes attention weights when output_attentions=True
tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 



LlamaModel is using LlamaSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [3]:
print(attention[0].shape)
print(attention[0][0].shape)
print(attention[0][0][0].shape)

torch.Size([1, 32, 123, 123])
torch.Size([32, 123, 123])
torch.Size([123, 123])


In [4]:
result = torch.zeros(attention[0].shape).to("cuda")
for layer in attention:
    result += layer

In [18]:
result[0][0][-1]

tensor([3.7840e+01, 1.0751e-01, 1.7387e-01, 1.6832e-01, 1.3956e-01, 4.2901e-02,
        5.3622e-02, 1.3439e-01, 9.7535e-02, 3.6954e-02, 3.6651e-02, 2.5741e-02,
        2.9334e-02, 7.1706e-02, 2.0036e-02, 6.1053e-02, 1.5970e-02, 1.1582e-02,
        3.0202e-02, 2.7284e-02, 2.6325e-02, 2.1142e-02, 8.9679e-02, 1.4984e-02,
        9.3856e-03, 1.9408e-02, 1.7252e-02, 4.5867e-02, 2.2013e-02, 5.3699e-02,
        2.1833e-02, 2.2869e-02, 4.5319e-02, 1.1987e-02, 1.4781e-01, 1.2118e-02,
        1.0493e-02, 2.8527e-02, 2.6911e-02, 1.5986e-02, 1.0016e-02, 1.6670e-02,
        2.3561e-02, 1.2790e-02, 1.1334e-02, 1.5565e-02, 7.7524e-03, 1.5345e-02,
        9.0209e-03, 4.7012e-02, 1.3786e-02, 1.1830e-02, 3.2980e-02, 1.5165e-02,
        1.8669e-02, 1.6376e-02, 1.0288e-02, 1.2649e-02, 5.5567e-03, 7.7424e-03,
        1.2250e-02, 1.5020e-02, 1.4751e-02, 8.7195e-03, 1.5985e-02, 2.6752e-02,
        3.8838e-02, 1.3985e-02, 9.7210e-03, 4.9957e-03, 1.2949e-02, 1.1111e-02,
        1.4243e-02, 4.9096e-02, 4.0397e-

In [11]:
result.shape

torch.Size([1, 32, 123, 123])

In [12]:
attention2 = tuple([result] + [i for i in attention[1:]])


In [66]:
attention2

(tensor([[[[48.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [45.7457,  2.2530,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [43.4800,  2.1382,  2.3829,  ...,  0.0000,  0.0000,  0.0000],
           ...,
           [38.7554,  0.7176,  0.6390,  ...,  1.6464,  0.0000,  0.0000],
           [39.9022,  0.4503,  0.4166,  ...,  1.4534,  1.6614,  0.0000],
           [38.3122,  1.1326,  0.5890,  ...,  0.7079,  2.3141,  1.5070]],
 
          [[48.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [45.1681,  2.8312,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [44.3584,  2.4422,  1.1989,  ...,  0.0000,  0.0000,  0.0000],
           ...,
           [41.8001,  0.9584,  0.5108,  ...,  0.7981,  0.0000,  0.0000],
           [41.2029,  0.6416,  0.5117,  ...,  1.0565,  1.2759,  0.0000],
           [39.5099,  0.8153,  0.5162,  ...,  0.4732,  1.5020,  2.0266]],
 
          [[48.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [4

In [67]:
head_view(attention2, tokens)

<IPython.core.display.Javascript object>

In [42]:
len(attention[0])

TypeError: len() of a 0-d tensor

In [ ]:
head_view(attention, tokens, html_action='return')
#model_view(attention, tokens, html_action='return', display_mode="light")